In [1]:
import pandas as pd

In [2]:
chalearn_path = "/home/vlados/datasets/chalearn/"
chalearn_labels_path = chalearn_path + "Info_devel_valid.txt"

In [3]:
chalearn_labels_df = pd.read_csv(chalearn_labels_path, sep="\t")

In [4]:
chalearn_labels_df.head()

,Set,Num,Lexicon,UserID,Date,MinDepth,MaxDepth,DepthRes,DepthAcc,Missing
0,devel,1,CanadaAviationGroundCirculation1,A,2011 10 05 16 26,801,1964,76,2,0
1,devel,2,RefereeWrestlingSignals1,I,2011 09 28 14 31,801,1968,71,1,0
2,devel,3,GangHandSignals1,L,2011 09 27 12 16,801,1506,95,1,0
3,devel,4,DivingSignals2,J,2011 09 26 09 01,801,1869,100,1,0
4,devel,5,GestunoDisaster,L,2011 10 11 15 16,824,1964,91,2,0


In [5]:
chlearn_diving_labels_df = chalearn_labels_df.loc[chalearn_labels_df["Lexicon"].isin(["DivingSignals1", "DivingSignals2"])].drop(chalearn_labels_df.loc[:,"UserID":], axis = 1)
chlearn_diving_labels_df

,Set,Num,Lexicon
3,devel,4,DivingSignals2
19,devel,20,DivingSignals1
22,valid,3,DivingSignals2
45,devel,26,DivingSignals2
54,devel,35,DivingSignals1
62,devel,43,DivingSignals2
71,devel,52,DivingSignals1
90,devel,71,DivingSignals1
92,devel,73,DivingSignals1
98,devel,79,DivingSignals2


In [6]:
import os

In [12]:
def find_dir(number, path):
    for dirname in os.listdir(path):
        splitted = dirname.split("-")
        if splitted[0] != "devel":
            continue
        if (int(splitted[1]) < number <= int(splitted[2])):
            subpath = os.path.join(path, dirname)
            for subdirname in os.listdir(subpath):
                subsplitted = subdirname.split("devel")
                if subsplitted[0] != "":
                    continue
                if int(subsplitted[1]) == number:
                    dest_path = os.path.join(subpath, subdirname)
                    for dest_file in os.listdir(dest_path):
                        if dest_file.split(".")[1] == "csv":
                            print(dest_path)
                            yield os.path.join(dest_path, dest_file)
                            

In [13]:
df = pd.DataFrame(columns=["video", "label"])
for df_path in find_dir(chlearn_diving_labels_df.iloc[0]["Num"], chalearn_path):
    df = pd.concat([df, pd.read_csv(df_path,names=["video", "label"])], ignore_index=True, sort=False)
df

/home/vlados/datasets/chalearn/devel-1-20/devel04
/home/vlados/datasets/chalearn/devel-1-20/devel04


,video,label
0,devel04_11,3
1,devel04_12,1
2,devel04_13,2 7
3,devel04_14,2
4,devel04_15,9 5 4
5,devel04_16,7
6,devel04_17,4 7 4 3
7,devel04_18,5
8,devel04_19,9 2
9,devel04_20,9 10 8 4


In [10]:
import numpy as np
import cv2
import os
import torch
import torch.nn as nn

os.sys.path.append('poseEstimation')
from poseEstimation.demo import infer_fast, VideoReader
from poseEstimation.modules.pose import Pose
from poseEstimation.modules.load_state import load_state
from poseEstimation.modules.keypoints import extract_keypoints, group_keypoints
from poseEstimation.models.with_mobilenet import PoseEstimationWithMobileNet

In [11]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"
print("Using device: " + DEVICE)
if torch.backends.cudnn.is_available():
    torch.backends.cudnn.enabled = True

SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

net = PoseEstimationWithMobileNet()
checkpoint = torch.load(
    "weights/checkpoint_iter_370000.pth", map_location='cpu')
load_state(net, checkpoint)

Using device: cuda


In [19]:
def infer(net, image_provider, height_size, cpu):
    net = net.eval()
    if not cpu:
        net = net.cuda()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts

    pose_sequence = []
    prediction = 0
    prediction_made = False

    for img in image_provider:
        heatmaps, pafs, scale, pad = infer_fast(
            net, img, height_size, stride, upsample_ratio, cpu)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(
                heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(
            all_keypoints_by_type, pafs)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (
                all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (
                all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            for kpt_id in range(num_keypoints):
                
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(
                        all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(
                        all_keypoints[int(pose_entries[n][kpt_id]), 1])
                else:
                    pose_keypoints[kpt_id, 0] = 0
                    pose_keypoints[kpt_id, 1] = 0
            pose = Pose(pose_keypoints, pose_entries[n][18])
            
            current_poses.append(pose)

        if (len(current_poses) > 0):
            pose_sequence.append(current_poses[0].keypoints.reshape([36]))

    return pose_sequence

In [35]:
column_names = [  "j0_x",  "j0_y", "j1_x", "j1_y" , "j2_x", "j2_y", "j3_x", "j3_y", "j4_x", "j4_y", "j5_x", "j5_y", "j6_x", "j6_y", "j7_x", "j7_y", "j8_x", "j8_y", "j9_x", "j9_y", "j10_x", "j10_y", "j11_x", "j11_y", "j12_x", "j12_y", "j13_x", "j13_y", 'j14_x', "j14_y", "j15_x", "j15_y", "j16_x", "j16_y", "j17_x", "j17_y" ]

In [20]:
frame_provider = VideoReader("/home/vlados/datasets/chalearn/devel-1-20/devel04/M_1.avi")
pose_sequence = infer(net, frame_provider, 256, False)

In [30]:
pose_sequence

[array([132,  80, 130, 110, 100, 108,  94, 150, 117, 165, 158, 114, 169,
        157, 167, 204, 115, 200,   0,   0,   0,   0, 152, 204, 158, 238,
          0,   0, 126,  75, 137,  75, 117,  76, 147,  76], dtype=int32),
 array([132,  80, 128, 112, 100, 110,  90, 150, 115, 170, 158, 114, 169,
        157, 167, 202, 115, 196,   0,   0,   0,   0, 152, 198, 158, 238,
          0,   0, 126,  75, 137,  75, 117,  76, 145,  76], dtype=int32),
 array([132,  80, 128, 112, 100, 110,  89, 150, 109, 176, 158, 114, 169,
        157, 167, 202, 115, 198, 113, 238,   0,   0, 152, 200, 156, 238,
          0,   0, 126,  75, 137,  75, 117,  76, 145,  76], dtype=int32),
 array([132,  80, 128, 112, 100, 110,  89, 153, 104, 185, 158, 114, 169,
        157, 167, 202, 115, 204,   0,   0,   0,   0, 152, 206, 158, 232,
          0,   0, 126,  75, 137,  75, 117,  76, 145,  76], dtype=int32),
 array([132,  80, 128, 112,  98, 112,  87, 155,  98, 196, 158, 114, 169,
        157, 167, 202, 113, 206,   0,   0,   0,   0

In [36]:
pd.DataFrame(pose_sequence, columns=column_names)

,j0_x,j0_y,j1_x,j1_y,j2_x,j2_y,j3_x,j3_y,j4_x,j4_y,...,j13_x,j13_y,j14_x,j14_y,j15_x,j15_y,j16_x,j16_y,j17_x,j17_y
0,132,80,130,110,100,108,94,150,117,165,...,0,0,126,75,137,75,117,76,147,76
1,132,80,128,112,100,110,90,150,115,170,...,0,0,126,75,137,75,117,76,145,76
2,132,80,128,112,100,110,89,150,109,176,...,0,0,126,75,137,75,117,76,145,76
3,132,80,128,112,100,110,89,153,104,185,...,0,0,126,75,137,75,117,76,145,76
4,132,80,128,112,98,112,87,155,98,196,...,0,0,126,75,137,75,119,76,147,78
5,132,80,128,114,98,112,87,157,94,198,...,0,0,126,75,137,75,119,76,147,76
6,132,80,128,112,98,112,85,155,90,200,...,0,0,126,75,137,75,117,76,147,76
7,132,80,128,112,98,112,85,155,85,200,...,0,0,126,75,137,75,117,76,145,76
8,132,80,128,114,98,112,85,153,85,198,...,0,0,126,75,137,75,115,76,145,76
9,132,80,128,114,98,112,85,155,85,200,...,0,0,126,75,137,75,115,76,145,76
